In [ ]:
import math
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
from board import Battleship

In [ ]:
from utils import display_heatmap

## Setup The Target Board

In [ ]:
board, solution = Battleship().create_board()

In [ ]:
display_heatmap(data=board)

## Create a Random Guess

In [ ]:
random_solution = np.random.randint(2, size=100).reshape((10, 10))

display_heatmap(data=random_solution)

## Define GA

In [ ]:
class GeneticAlgorithm(object):
    """Generate GA for Battleship problem."""
    
    def __init__(self, num_generations=100):
        
        self.num_generations = num_generations
    
    def generate_population(self, num_individuals, genes):
        
        # create dataframe for gene pool
        df_population = pd.DataFrame(columns=['Sequence', 'Chromosome', 'Generation', 'Birth', 'Fitness', 'Parents'])
        
        i = 0
        while i < num_individuals:
            
            individual = {
                'Sequence': i+1,
                'Chromosome': ''.join(str(x) for x in list(np.random.randint(2, size=genes))),
                'Generation': 1,
                'Birth': 'Random',
                'Parents': 0
            }
            
            # check for uniqueness and add to gene pool
            if individual['Chromosome'] not in df_population['Chromosome']:
                df_population = df_population.append(individual, ignore_index=True)
                i += 1
                
        # return the generation
        return generation

    def evaluate_fitness(self):
        pass
    
    def crossover(self, next_generation, num_crossover):
        
        # get generation attributes
        last_generation = next_generation['Generation'].max()
        last_sequence = next_generation['Sequence'].max()
        num_elites = next_generation['Birth'].value_counts()['Elitism']
        
        i = 0
        while i < num_crossover:
            
            # create crossover chromosome
            individual = {
                'Generation': last_generation,
                'Birth': 'Crossover',
                'Elite': False
            }
            
            # select random crossover as new parents
            parent_indices = np.random.choice(num_elites, 2, replace=False)
            individual['Parents'] = np.array(next_generation['Sequence'].values)[parent_indices]
            parents = np.array(next_generation['Chromosome'].values)[parent_indices]
            
            # create crossover bit
            crossover_bit = np.random.randint(len(parents[0]))
            
            # create crossover children from parent and crossover bits
            crossover = []
            crossover.append(parents[0][0:crossover_bit] + parents[1][crossover_bit:len(parents[1])])
            crossover.append(parents[1][0:crossover_bit] + parents[0][crossover_bit:len(parents[0])])
            
            # add crossover to population
            individual['Chromosome'] = crossover[0]
            individual['Sequence'] = last_sequence + i + 1
            next_generation = next_generation.append(individual, ignore_index=True)
            
            individual['Chromosome'] = crossover[1]
            individual['Sequence'] = last_sequence + i + 2
            next_generation = next_generation.append(individual, ignore_index=True)

            i += 1

        # return the generation
        return next_generation

    def mutate(self, next_generation, num_mutants, bit_flip_rate):

        # get generation attributes
        last_generation = next_generation['Generation'].max()
        last_sequence = next_generation['Sequence'].max()
        num_elites = next_generation['Birth'].value_counts()['Elitism']
        
        i = 0
        while i < num_mutants:
            
            # create mutant chromosome
            for i in range(num_individuals):

                individual = {
                    'Sequence': last_sequence + i + 1,
                    'Generation': last_sequence,
                    'Birth': 'Mutation',
                    'Elite': False
                }
                
                # select random elite as new parent
                parent_index = np.random.choice(num_elites)
                individual['Parents'] = list(next_generation['Sequence'].values)[parent_index]
                parent = list(next_generation['Chromosome'].values)[parent_index]
            
            # create array of random bit flips
            bit_flip_array = np.random.choice(2, len(parent), p=[1 - bit_flip_rate, bit_flip_rate])
            bits_to_flip = ''.join(str(x) for x in list(bit_flip_array.flatten()))

            # create mutant child from parent and flip bits from array
            mutant = ''
            for j in range(len(bits_to_flip)):
                if not int(bits_to_flip[j]):
                    mutant += parent[j]
                else:
                    mutant += str(abs(int(parent[j]) - 1))
                    
            # check for uniqueness and add to population
            individual['Chromosome'] = mutant
            if individual['Chromosome'] not in next_generation['Chromosome']:
                next_generation = next_generation.append(individual, ignore_index=True)
                i += 1
                
        # return the generation
        return next_generation

    def create_next_generation(self, population, solution, elite_rate, stop_limit):
            
        next_generation = population.copy()
        num_individuals = next_generation.shape[0]

        # create generations until fitness criteria is achieved
        while population['Fitness'].max() < stop_limit:
            
            # select elites with elite rate
            next_generation = self.select_elites(next_generation)
            
            # add crossover pairs to generation
            crossover_rate = elite_rate / 2
            num_crossover = int(crossover_rate * num_individuals)
            next_generation = self.crossover(next_generation, num_crossover)
            
            # add mutants to generation
            mutant_rate = 0.60
            bit_flip_rate = 0.01
            num_mutants = int(mutant_rate * num_individuals)
            next_generation = self.mutate(next_generation, num_mutants, bit_flip_rate)
            
            # fill the rest of the generaion with random chromosomes for diversity
            next_generation = self.fill_random(next_generation, num_individuals, genes=100)
            
            # evaluate fitness
            next_generation['Fitness'] = self.evaluate_fitness()
            
            # assign elite rate
            elite_rate = 0.20
            next_generation = self.assign_elites(next_generation, elite_rate)
            
            # add next generation to population
            population = population.append(next_generation)

        return population
    
    def reproduce(self, num_individuals, solution, genes):
        
        # initialize the first random population
        population = self.generate_population(num_individuals, genes)
        
        # evaluate the fitness
        population['Fitness'] = self.evaluate_fitness()
        
        # assign elite rate
        elite_rate = 0.20
        population = self.assign_elites(population, elite_rate)
        
        # create successive generations until termination criteria is met
        population = self.create_next_generation()
        population = population.set_index['Sequence']
        
        return population

---